In [1]:
from pathlib import Path
from types import SimpleNamespace
import tensorflow as tf

import wandb

from data_loader import DataLoader
from utils.callbacks import TensorBoard
from utils.util import *
from utils.args_loader import load_model_config

2022-09-05 18:48:47.099246: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-05 18:48:47.099285: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_path = Path("/mnt/disks/KITTI/small/")
data_path.exists()

True

In [3]:
arg = SimpleNamespace(model="squeezesegv2",
                      config="squeezesegv2kitti",
                      data_path=data_path,
                      train_dir="../output",
                      epochs=10)   

config, model = load_model_config(arg.model, arg.config)

2022-09-05 18:48:49.155193: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-05 18:48:49.156015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-05 18:48:49.156096: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-09-05 18:48:49.156165: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-09-05 18:48:49.156234: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [4]:
train_dl = DataLoader("train", arg.data_path, config).write_tfrecord_dataset().read_tfrecord_dataset()
val_dl = DataLoader("val", arg.data_path, config).write_tfrecord_dataset().read_tfrecord_dataset()

TFRecord exists at /mnt/disks/KITTI/small/train.tfrecord. Skipping TFRecord writing.
TFRecord exists at /mnt/disks/KITTI/small/val.tfrecord. Skipping TFRecord writing.


## Log validation dataset

In [11]:
def resize(img, factor=1):
    return tf.image.resize(img, [img.shape[0]*factor, img.shape[1]*factor]).numpy()

In [17]:
def _create_row(lidar_input, label, class_color_map):
    "Create tuple of images and points"
    label = label[:, :]
    label_image = class_color_map[label.numpy().reshape(-1)].reshape([label.shape[0], label.shape[1], 3])
    depth_image = lidar_input.numpy()[:, :, [4]]
    intensity = lidar_input.numpy()[:, :, [3]]
    points = lidar_input.numpy()[:, :, :3]
    points_rgb = np.concatenate([points, (255*label_image).astype(int)], axis=-1).reshape(-1, 6)
    
    depth_image, label_image, intensity_image  = map(resize, [depth_image, label_image, intensity])
    return (wandb.Image(label_image), 
            wandb.Image(depth_image), 
            wandb.Image(intensity_image), 
            wandb.Object3D({"type": "lidar/beta", "points":points_rgb}),
           )

In [13]:
def log_input_data(lidar_input, label, class_color_map):
    "Log to media panel"
    label_image, depth_image, intensity_image, points_rgb = _create_row(lidar_input, 
                                                                        label, s
                                                                        class_color_map)
    # log 2 wandb
    wandb.log({'Images/Label Image': label_image})
    wandb.log({'Images/Depth Image': depth_image})
    wandb.log({'Images/Intensity Image': intensity_image})
    wandb.log({"Images/3D": points_rgb})

In [18]:
table_data = []
n=20
for i, ((lidar_input, _), label, _) in enumerate(val_dl):
    if i < n:
        table_data.append(_create_row(lidar_input[0], label[0], model.CLS_COLOR_MAP))

In [19]:
table = wandb.Table(columns=["Label Image", "Depth Image", "Intensity Image", "LiDAR"],
                    data=table_data)

In [20]:
with wandb.init(project="small_kitti", entity="av-team", job_type="log_dataset"):
    wandb.log({"validation_ds":table})